<a href="https://colab.research.google.com/github/simran-sai/ALLTOGO-Medihelp/blob/main/ALLTOGO_Medihelp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets geopy google-auth google-auth-oauthlib google-auth-httplib2 gspread SpeechRecognition pyttsx3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 21.9 MB/s eta 0:00:00


In [ ]:
import requests
import sqlite3
from transformers import AutoModelForCausalLM, AutoTokenizer
from geopy.geocoders import Nominatim
import geocoder


In [ ]:
import pandas as pd
from datasets import Dataset

# Load MedQuAD dataset
data_path = "/path_to_medquad/medquad.csv" # Make sure this path is correct and the file exists
df = pd.read_csv(data_path)

# Assuming the dataset has two columns: "question" and "answer"
# Prepare the dataset for BioT5 fine-tuning
def preprocess_medquad(examples):
    inputs = ["question: " + q for q in examples['question']]
    targets = [a for a in examples['answer']]
    return {'input_ids': inputs, 'labels': targets}

# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(df)
dataset = dataset.map(preprocess_medquad, batched=True)

FileNotFoundError: [Errno 2] No such file or directory: '/path_to_medquad/medquad.csv'

In [ ]:
conn = sqlite3.connect('medibot.db')
cursor = conn.cursor()

# Create users, appointments, and feedback tables
cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    username TEXT UNIQUE NOT NULL,
                    password TEXT NOT NULL)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS appointments (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    username TEXT NOT NULL,
                    doctor_name TEXT NOT NULL,
                    appointment_date TEXT NOT NULL)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS feedback (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    username TEXT NOT NULL,
                    feedback TEXT NOT NULL)''')

conn.commit()

In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 15.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load BioGPT
model_name = "microsoft/BioGPT-Large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def get_medical_assistance_biogpt(query):
    inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True)
    attention_mask = inputs.attention_mask
    pad_token_id = tokenizer.eos_token_id

    outputs = model.generate(inputs.input_ids, attention_mask=attention_mask, max_length=150, do_sample=True, temperature=0.7, pad_token_id=pad_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.strip()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.29G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
def get_user_location():
    g = geocoder.ip('me')  # Retrieves location using IP address
    if g.ok:
        return g.latlng
    else:
        return None

In [ ]:
# Function to find nearby doctors using OpenStreetMap Overpass API
def find_nearby_doctors(user_location, radius=5000):
    lat, lng = user_location
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    node
      ["amenity"="doctors"]
      (around:{radius},{lat},{lng});
    out body;
    """
    response = requests.get(overpass_url, params={'data': overpass_query})
    data = response.json()

    doctors = []
    for element in data['elements']:
        doctor = {
            'name': element['tags'].get('name', 'N/A'),
            'address': f"{element['lat']}, {element['lon']}",
        }
        doctors.append(doctor)

    return doctors if doctors else None

In [ ]:
# User Authentication
def signup(username, password):
    try:
        cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, password))
        conn.commit()
        return "Signup successful!"
    except sqlite3.IntegrityError:
        return "Username already exists."

def login(username, password):
    cursor.execute('SELECT * FROM users WHERE username=? AND password=?', (username, password))
    user = cursor.fetchone()
    if user:
        return "Login successful!"
    else:
        return "Invalid username or password."

In [ ]:
def book_appointment(username, doctor_name, appointment_date):
    cursor.execute('INSERT INTO appointments (username, doctor_name, appointment_date) VALUES (?, ?, ?)',
                   (username, doctor_name, appointment_date))
    conn.commit()
    return f"Appointment with Dr. {doctor_name} on {appointment_date} has been booked."


In [ ]:
def submit_feedback(username, feedback):
    cursor.execute('INSERT INTO feedback (username, feedback) VALUES (?, ?)', (username, feedback))
    conn.commit()
    return "Thank you for your feedback!"

In [ ]:
def main():
    print("Welcome to MediBot in Google Colab!")
    while True:
        action = input("Do you want to login or signup? (login/signup/exit): ").lower()

        if action == "exit":
            break

        username = input("Enter username: ")
        password = input("Enter password: ")

        if action == "login":
            login_status = login(username, password)
            print(login_status)
            if login_status == "Login successful!":
                while True:
                    user_input = input("Ask a medical question or type 'find doctor' (or type 'logout' to exit): ").lower()
                    if user_input == "logout":
                        break
                    elif user_input == "find doctor":
                        user_location = get_user_location()
                        if user_location:
                            doctors = find_nearby_doctors(user_location)
                            if doctors:
                                print("Doctors near you:")
                                for doctor in doctors:
                                    print(f"Name: {doctor['name']}, Address: {doctor['address']}")
                            else:
                                print("No doctors found nearby.")
                        else:
                            print("Could not retrieve your location.")
                    else:
                        advice = get_medical_assistance_biogpt(user_input)
                        print(f"Medical Advice: {advice}")
        elif action == "signup":
            signup_status = signup(username, password)
            print(signup_status)

if __name__ == "__main__":
    main()

Welcome to MediBot in Google Colab!
Login successful!
Medical Advice: how to do intermittent fasting effectively? < / FREETEXT > < / TITLE > ▃ < ABSTRACT > < FREETEXT > Intermittent fasting has been proposed as a new approach for weight loss and treatment of metabolic diseases such as type 2 diabetes. However, the mechanisms of action of intermittent fasting are not well understood. A review of published data on intermittent fasting is presented here. We discuss the current knowledge on the effects of intermittent fasting on glucose and lipid metabolism, inflammation, body weight and body composition, energy expenditure and physical activity, and mood states. Intermittent fasting has been shown to improve glucose tolerance and lipid metabolism, reduce body weight and fat, increase energy expenditure and physical activity, and improve mood states. The effects of intermittent fasting are consistent among all studies. We
Doctors near you:
Name: Fresious Medical Care, Address: 45.6052333, 